In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import date, timedelta, datetime
%matplotlib inline
import seaborn as sns

In [5]:
filename = "ECOW.csv"
STORAGE_FOLDER = ""

In [6]:
data = pd.read_csv("ECOW.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'ECOW.csv'

In [7]:
data.tail(10)

NameError: name 'data' is not defined

In [8]:
data.shape

NameError: name 'data' is not defined

In [10]:
# to print the full summary
data.info()

NameError: name 'data' is not defined

In [ ]:
# knowling number of non-missing values for each variable
data.isnull().sum()

In [ ]:
data = data.drop(labels=2481, axis=0)

In [ ]:
data.shape

In [ ]:
train_df = data.sort_values(by=['Date']).copy()

# We safe a copy of the dates index, before we need to reset it to numbers
date_index = train_df.index

In [ ]:
X = data.drop(columns=['Date', 'Value'], axis=1)
Y = data['Value']

In [ ]:
X_train = X.loc[:2000, : ]
Y_train = Y.loc[:2000]
X_test = X.loc[2000:, :]
Y_test = Y.loc[2000:]

In [ ]:
print(X.shape, X_train.shape, X_test.shape)
print(Y.shape, Y_train.shape, Y_test.shape)

In [ ]:
scaler = MinMaxScaler()
#scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(X_train)

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

classifier = Sequential()

classifier.add(Dense(units = 128, activation = 'relu', input_dim=20))
classifier.add(Dropout(rate = 0.5))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(rate = 0.5))
#classifier.add(Dense(units = 256, activation = 'relu'))
#classifier.add(Dropout(rate = 0.1))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(rate = 0.5))
classifier.add(Dense(units = 1, activation = 'relu'))
classifier.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics=['mean_absolute_error'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=200)
classifier.summary()

In [ ]:
history = classifier.fit(X_train,Y_train,epochs=150, batch_size=60, validation_data=(X_test, Y_test), callbacks=[es])

In [ ]:
test_loss, test_acc = classifier.evaluate(X_test, Y_test, verbose=2)

In [ ]:
plt.figure(figsize=(9,9))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.ylim([0,1.0])
plt.legend(loc='lower right')

In [ ]:
predictions = classifier.predict(X_test)
def mean_absolute_percentage_error(Y_test, predictions):
  Y_test, predictions = np.array(Y_test), np.array(predictions)
  return np.mean(np.abs((Y_test - predictions) / Y_test)) * 100
mape_ann=mean_absolute_percentage_error(Y_test, predictions)

rmse_ann = np.sqrt(mean_squared_error(Y_test, predictions))
print('Test RMSE: %.3f' % rmse_ann)
mape_ann = mape_ann
print('Test MAPE: %.3f' % mape_ann + '%')
PP_ann = 1-rmse_ann/Y_train.std()
print('PP: %.3f' % PP_ann)